In [465]:
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB

In [466]:
def MaxMinDiversity(subset: np.ndarray, complete: np.ndarray) -> float:
    """
    Calculate Max-Min Diversity, the minimum distance between all pairs of points in the given subset.
    
    Parameters:
        subset: Subset of points
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_dist: Max-Min Diversity, the minimum distance between all pairs of points in the subset
    """
    if len(subset) < 2:
        return 0
    
    n = len(subset)
    min_dist = float('inf')
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = complete[subset[i]][subset[j]]
            min_dist = min(min_dist, dist)
    
    return min_dist

In [467]:
def FairRadius(k: int, complete: np.ndarray) -> np.ndarray:
    """
    Calculate Fair Radius for all points in the dataset.

    Parameters:
        k: Integer parameter k, used to calculate minimum number of points each point needs to cover ⌈n/k⌉
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        fair_radius: Fair Radius for all points in the dataset
    """
    n = complete.shape[0]
    min_points = int(np.ceil(n / k))
    fair_radius = []

    for i in range(n):
        distances = complete[i]
        sorted_distances = np.sort(distances)
        point_radius = sorted_distances[min_points-1]
        fair_radius.append(point_radius)
        
    return np.array(fair_radius)


In [468]:
def CriticalRegion(alpha: float, fair_radius: np.ndarray, complete: np.ndarray) -> tuple[np.ndarray, dict]:
    """
    Generate the Critical Regions by finding a set of centers that cover all points.
    
    Parameters:
        alpha: Fairness parameter
        fair_radius: Array containing fair radius values for all points
        complete: Complete graph adjacency matrix representing distances between all pairs of points
        
    Returns:
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping each center to its covered points
    """
    covered_points = set()
    n = len(fair_radius)
    points = set(range(n))
    selected_centers = []
    critical_regions = {}

    while covered_points != points:
        minus = points - covered_points
        c = min(minus, key=lambda x: fair_radius[x])
        selected_centers.append(c)
        for point in minus:
            if complete[point][c] <= 2 * alpha * fair_radius[point]:
                covered_points.add(point)
    selected_centers = np.array(selected_centers)
    print(covered_points)

    for center in selected_centers:
        r_c = fair_radius[center]
        distances = complete[center]
        points_in_circle = np.array([i for i, dist in enumerate(distances) if dist <= alpha * r_c])
        critical_regions[center] = points_in_circle

    return selected_centers, critical_regions


In [469]:
def IFDM(k: int, complete: np.ndarray, epsilon: float, beta: float, selected_centers: np.ndarray, critical_regions: dict) -> tuple[np.ndarray, tuple[tuple[np.ndarray, int], ...], np.ndarray, dict]:
    """
    Construct an instance of k-clustering under partition matroid constraint corresponding to the given instance of alpha-fair k-clustering.
    
    Parameters:
        k: Target number of clusters
        alpha: Fairness parameter
        complete: Complete graph adjacency matrix with distances
        epsilon: Accuracy parameter (< 1/2)
        beta: Approximation guarantee parameter (≤ 1)
        selected_centers: Centers of critical regions
        critical_regions: Dictionary mapping centers to covered points
        
    Returns:
        P_prime: Augmented point set with duplicated points
        centers_info: Tuple containing center selection info
        d_prime: Modified graph adjacency matrix with distances
        corresponding: The dictionary for getting the corresponding point in originial data
    """
    n = complete.shape[0]
    P_0 = list(range(n))
    P_1 = list(range(n))
    current_max_id = n
    
    B_copies = {}
    corresponding = {}
    for center in selected_centers:
        copies = []
        for point in critical_regions[center]:
            corresponding[current_max_id] = point
            copies.append(current_max_id)
            P_1.append(point)
            current_max_id += 1
        B_copies[center] = np.array(copies)

    P_prime = np.arange(current_max_id)
    
    k_i = {center: 1 for center in selected_centers}
    
    k_0 = k - len(selected_centers)

    delta = np.min(complete[complete > 0])

    n_prime = len(P_prime)
    d_prime = np.zeros((n_prime, n_prime))

    for i in range(n_prime):
        for j in range(n_prime):
            if i == j:
                d_prime[i][j] = 0
            elif P_1[i] != P_1[j]:
                d_prime[i][j] = round(complete[P_1[i]][P_1[j]], 10)
            else:
                d_prime[i][j] = round(epsilon * beta * delta, 10)
                
    return P_prime, ((P_0, k_0), *((B_copies[c], k_i[c]) for c in selected_centers)), d_prime, corresponding


In [470]:
def distancePS(centerSet: np.ndarray, i: int, complete: np.ndarray) -> float:
    """
    Returns the distance between a certain point and a certain set.
    
    Parameters:
        centerSet: A numpy array containing confirmed center indexes
        i: The index of any point
        complete : Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        min_distance: The distance between point and center set
    """
    min_distance = float("inf")
    for center in centerSet:
        distance = complete[center][i]
        if (distance < min_distance):
            min_distance = distance
    
    return min_distance

In [471]:
def GMM(points_index: np.ndarray, k: int, complete: np.ndarray) -> np.ndarray:
    """
    Returns indexes of k centers after running GMM Algorithm.
    
    Parameters: 
        points_index: The indexes of data
        k: A decimal integer, the number of centers
        complete: Complete graph adjacency matrix containing distances between all pairs of points
        initial: An initial set of elements
    
    Returns:
        centers: A numpy array with k indexes as center point indexes
    """
    centers = []
    initial_point_index = random.choice(points_index)
    centers.append(initial_point_index)
    while (len(centers) < k):
        max_distance = 0
        max_distance_vector_index = None
        for i in points_index:
            distance = distancePS(centers, i, complete)
            if distance > max_distance:
                max_distance = distance
                max_distance_vector_index = i
        centers.append(max_distance_vector_index)
    centers = np.array(centers)

    return centers

In [472]:
def ILP(n, E, k, color_constraints, color_sets):
    """
    Returns an ILP solution for FMMD-S.
    
    Parameters:
        n: The number of points
        E: The undirected graph generated by FMMD-S
        k: A decimal integer, the number of centers 
        color_constraints: The groups that the data are divided into
        color_sets: The points in different groups
    
    Returns:
        selected_nodes: A numpy array containing selected elements that maximize the minimum pairwise distance    
    """
    model = gp.Model("ILP")

    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)

    eid = 0
    for e in E:
        model.addConstr(x[e[0]] + x[e[1]] <= 1, f"edge_{str(eid)}")
        eid += 1

    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= k, "max_selection")

    for c, number in color_constraints.items():
        nodes_in_color = color_sets[c]
        if number > 1:
            model.addConstr(gp.quicksum(x[i] for i in nodes_in_color) <= number, f"number_color_{c}")
        else:
            model.addConstr(gp.quicksum(x[i] for i in nodes_in_color) == number, f"number_color_{c}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        selected_nodes = np.array([i for i in range(n) if x[i].x > 0.5])
        return selected_nodes
    else:
        raise ValueError("No optimal solution found.")
        

In [473]:
def FMMDS(sets: tuple, k: int, error: float, complete: np.ndarray) -> np.ndarray:
    """
    Returns a subset with high Max-min diversity under partition matroid constraint.
    
    Parameters:
        sets: A tuple containing partitioned sets returned by IFDM function
        k: A decimal integer, the number of elements to be selected
        error: A float number indicating the error parameter
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        solution: A numpy array containing selected elements that maximize the minimum pairwise distance
    """
    amount = complete.shape[0]
    complete_array = np.arange(amount)
    U_gmm = GMM(complete_array, k, complete)
    critical_number = len(sets)
    U_c = []
    for i in range(critical_number):
        intersection = np.intersect1d(sets[i][0], U_gmm)
        if len(intersection) > 0:
            U_c.append(intersection)
        else:
            random_element = np.random.choice(sets[i][0])
            U_c.append(np.array([random_element]))
            
    distance_p = 2 * MaxMinDiversity(U_gmm, complete)
    print("distance_p:", distance_p)

    S = []
    while len(S) == 0:
        for c in range(critical_number):  
            max_div_uc_v = 0
            for v in sets[c][0]:
                new_div_uc_v = MaxMinDiversity(np.union1d(U_c[c], [v]), complete)
                if new_div_uc_v > max_div_uc_v:
                    max_div_uc_v = new_div_uc_v
            print(max_div_uc_v)
            print(distance_p)
            while len(U_c[c]) < k and max_div_uc_v >= distance_p:
                max_distance = 0
                max_distance_vector_index = 0
                for i in sets[c][0]:
                    distance = distancePS(U_c[c], i, complete)
                    if distance > max_distance:
                        max_distance = distance
                        max_distance_vector_index = i
                U_c[c] = np.append(U_c[c], max_distance_vector_index)

        V_p = np.concatenate(U_c)
        index2V_p = dict()
        V_p2index = dict()
        for index, point in enumerate(V_p):
            index2V_p[index] = point
            V_p2index[point] = index

        E = []
        for i in range(len(V_p)):
             for j in range(i + 1, len(V_p)):
                  print("complete:", complete[V_p[i]][V_p[j]])
                  if complete[V_p[i]][V_p[j]] < distance_p / 2:
                       
                       E.append((i, j))
        print(E)

        color_constraints = dict()
        color_sets = dict()
        for group in range(critical_number):
            color_constraints[group] = sets[group][1]
            color_sets_original = np.intersect1d(sets[group][0], V_p)
            color_sets_ilp = []
            for point in color_sets_original:
                color_sets_ilp.append(V_p2index[point])
                print(color_sets_ilp)
            color_sets[group] = np.array(color_sets_ilp)
            
        print(color_constraints)
        print(color_sets)

        solution = ILP(len(V_p), E, k, color_constraints, color_sets)
        if len(solution) < k:
             distance_p = (1 - error) * distance_p
        else:
             S = solution

    for i in range(len(S)):
        S[i] = index2V_p[i]
    
    return S

In [474]:
def SOL(S: np.ndarray, corresponding: dict, complete: np.ndarray) -> np.ndarray:
    """
    Returns a np.ndarray with high Max-min diversity in the original data.
    
    Parameters:
        S: The solution from FMMDS
        corresponding: The dictionary for getting the corresponding point in originial data
        complete: Complete graph adjacency matrix containing distances between all pairs of points
    
    Returns:
        solution: A numpy array containing selected elements that maximize the minimum pairwise distance
    """
    solution = []
    for point in S:
        if point < complete.shape[0]:
            solution.append(point)
        elif corresponding[point] not in solution:
            solution.append(corresponding[point])
        else:
            continue
    solution = np.array(solution)

    return solution

In [475]:
def Examine(complete: np.ndarray, solution: np.ndarray, fair_radius: np.ndarray, alpha: float, gamma: float) -> bool:
    
    points = np.arange(complete.shape[0])
    for point in points:
        if distancePS(solution, point, complete) > alpha * gamma * fair_radius[point]:
            return False
    
    return True

In [476]:
complete = np.load("dataset/adult_complete.npy")
complete

array([[0.        , 0.03521865, 0.0360143 , ..., 0.03676195, 0.03358565,
        0.037072  ],
       [0.03521865, 0.        , 0.03164265, ..., 0.03761507, 0.02821316,
        0.03421197],
       [0.0360143 , 0.03164265, 0.        , ..., 0.02900964, 0.00482017,
        0.0067394 ],
       ...,
       [0.03676195, 0.03761507, 0.02900964, ..., 0.        , 0.02602977,
        0.03397402],
       [0.03358565, 0.02821316, 0.00482017, ..., 0.02602977, 0.        ,
        0.00952895],
       [0.037072  , 0.03421197, 0.0067394 , ..., 0.03397402, 0.00952895,
        0.        ]])

In [477]:
complete.shape[0]

1000

In [478]:
len(complete)

1000

In [479]:
K = 5
alpha = 1

In [480]:
b = GMM(np.arange(1000), K, complete)

In [481]:
MaxMinDiversity(b, complete)

0.12642977654948243

In [482]:
fairradius = FairRadius(K, complete)
# fairradius

In [483]:
critical = CriticalRegion(alpha, fairradius, complete)
critical[0].shape

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

(1,)

In [484]:
for i in range(complete.shape[0]):
    if distancePS(critical[0], i, complete) > 2 * alpha * fairradius[i]:
        print(False)

In [485]:
# set(critical[1][critical[0][0]]) & set(critical[1][critical[0][1]])

In [486]:
ifdm = IFDM(K, complete, 0.1, 0.95 / 5, critical[0], critical[1])

In [487]:
ifdm[2].shape

(1200, 1200)

In [488]:
# for i in range(1150):
#     for j in range(1150):
#         if ifdm[2][i][j] != ifdm[2][j][i]:
#             print(False)
#             break

In [489]:
# for i in range(1150):
#     for j in range(1150):
#         if ifdm[2][i][j] < 0:
#             print(False)
#             break
#         if ifdm[2][i][j] == 0:
#             if i != j:
#                 print(False)
#                 break

In [490]:
# for i in range(1150):
#     for j in range(1150):
#         for k in range(1150):
#             if ifdm[2][i][j] > ifdm[2][i][k] + ifdm[2][j][k] + 1e-8:
#                 print(False)
#                 print(ifdm[2][i][j], ifdm[2][i][k] + ifdm[2][j][k])

In [491]:
ifdm[0].shape

(1200,)

In [492]:
solution = FMMDS(ifdm[1], K, 0.05, ifdm[2])

distance_p: 0.239894464
0.119947232
0.239894464
0.0188727807
0.239894464
complete: 0.1812372086
complete: 0.2511069068
complete: 0.2282710886
complete: 0.1467777509
complete: 0.1806259551
complete: 0.175223493
complete: 0.119947232
complete: 0.3204982216
complete: 0.0180492725
complete: 0.2144606028
complete: 0.3666723617
complete: 0.1740576418
complete: 0.3379578148
complete: 0.1207707728
complete: 0.3212773112
[(1, 5)]
[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[5]
{0: 4, 1: 1}
{0: array([0, 1, 2, 3, 4]), 1: array([5])}
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10200H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 6 columns and 14 nonzeros
Model fingerprint: 0x4335ef3e
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+0

In [493]:
solution

array([  8, 341, 360, 384, 648])

In [494]:
real_sol = SOL(solution, ifdm[3], complete)

In [495]:
real_sol

array([  8, 341, 360, 384, 648])

In [496]:
MaxMinDiversity(real_sol, complete)

0.11994723203753908

In [497]:
MaxMinDiversity(b, complete)

0.12642977654948243

In [498]:
Examine(complete, real_sol, fairradius, alpha, 3)

True

In [499]:
np.min(complete[complete > 0])

6.227537266812258e-05